In [11]:
from mmvae_hub.base.experiment_vis.utils import *
from mmvae_hub.base.utils.MongoDB import MongoDatabase
import os
from pathlib import Path
from pprint import pprint
import pandas as pd

In [12]:
exp_db = MongoDatabase(training=False)
experiments = exp_db.connect()

[14:33:51 CEST] Connecting to database.


In [13]:
print('There are currently ', experiments.count_documents({}), 'experiments in the database.')

There are currently  8 experiments in the database.


In [37]:
df = pd.DataFrame()
# Find all experiments with method joint_elbo
# for exp in experiments.find({'flags.method':'joint_elbo'}):
for exp in experiments.find({}):
    last_epoch = max(exp['epoch_results'])
    last_epoch_results = exp['epoch_results'][last_epoch]['test_results']
    results_dict = {**exp['flags'],'end_epoch': last_epoch, '_id': exp['_id']}

    scores = []
    scores_lr = []
    scores_gen = []

    # get lr_eval results
    for key, val in last_epoch_results['lr_eval'].items():
        results_dict[f'lr_eval_{key}'] = val['accuracy']
        scores.append(val['accuracy'])
        scores_lr.append(val['accuracy'])

    # get gen_eval results
    for key, val in last_epoch_results['gen_eval'].items():
        key = key.replace('digit_', '')
        results_dict[f'gen_eval_{key}'] = val
        scores.append(val)
        scores_gen.append(val)


    results_dict['score'] = sum(scores)
    results_dict['score_lr'] = sum(scores_lr)
    results_dict['score_gen'] = sum(scores_gen)
    df = df.append(results_dict, ignore_index=True)


In [38]:
show_cols = ['score','method', 'end_epoch', 'class_dim', 'beta','_id', 'score_lr', 'score_gen']
df[show_cols].sort_values(by=['score'], ascending=False)

,score,method,end_epoch,class_dim,beta,_id,score_lr,score_gen
7,9.847467,moe,99,128.0,0.5,polymnist_moe_2021_05_09_01_46_10_858142,3.542500,6.304967
6,9.830167,joint_elbo,99,64.0,0.5,polymnist_joint_elbo_2021_05_08_05_21_05_683057,3.762967,6.067200
0,8.623167,joint_elbo,99,64.0,2.5,polymnist_joint_elbo_2021_05_04_09_21_34_487039,3.604533,5.018633
5,6.461467,moe,99,64.0,0.5,polymnist_moe_2021_05_07_09_09_37_528857,2.374600,4.086867
4,6.352133,joint_elbo,99,32.0,0.5,polymnist_joint_elbo_2021_05_06_12_59_58_785343,2.550250,3.801883
3,4.797233,moe,99,32.0,0.5,polymnist_moe_2021_05_05_17_04_47_279351,1.459100,3.338133
2,3.063350,joint_elbo,0,64.0,2.5,polymnist_joint_elbo_2021_05_05_16_43_26_236427,0.896183,2.167167
1,2.859983,planar_mixture,99,64.0,2.5,polymnist_planar_mixture_2021_05_04_09_21_38_7...,0.730917,2.129067
